In [10]:
from sympy import *
import numpy as np
from itertools import product, combinations_with_replacement, chain
from operator import itemgetter, add
from math import factorial
from multiprocessing import Pool, cpu_count
x,c = symbols('x c')

In [11]:
cpu_count()

8

In [7]:
sympify("log(exp(Symbol('x')))")

log(exp(x))

In [8]:
1558**2

2427364

In [9]:
c = 1214461
nc = 2427364
s = 1558
2*c+3*nc+6*s

9720362